# Day 2

In [1]:
import $ivy.`org.scalaz::scalaz-core:7.2.22`
import scalaz._
import Scalaz._

import $ivy.$                               

import scalaz._

import Scalaz._

## Functor

https://github.com/scalaz/scalaz/blob/series/7.3.x/core/src/main/scala/scalaz/Functor.scala

```scala
trait Functor[F[_]]  { self =>
  /** Lift `f` into `F` and apply to `F[A]`. */
  def map[A, B](fa: F[A])(f: A => B): F[B]

  ...
}
```

```scala
trait FunctorOps[F[_],A] extends Ops[F[A]] {
  implicit def F: Functor[F]
  ////
  import Leibniz.===

  final def map[B](f: A => B): F[B] = F.map(self)(f)
  
  ...
}
```

In [2]:
List(1, 2, 3) map {_ + 1}

res1: List[Int] = List(2, 3, 4)

In [3]:
(1, 2, 3) map {_ + 1}

res2: (Int, Int, Int) = (1, 2, 4)

## Functor としての関数

In [4]:
val res = ((x: Int) => x + 1) map {_ * 7}

res: Int => Int = scalaz.std.FunctionInstances$$anon$3$$Lambda$3014/956417547@2ffa0fef

In [5]:
res(3)

res4: Int = 28

In [6]:
(((_: Int) * 3) map {_ + 100}) (1)

res5: Int = 103

In [7]:
List(1, 2, 3) map {3*}

res6: List[Int] = List(3, 6, 9)

In [8]:
val res = Functor[List].lift {(_: Int) * 2}

res: List[Int] => List[Int] = scalaz.Functor$$Lambda$3053/1267835070@618f126d

In [9]:
res(List(3))

res8: List[Int] = List(6)

In [10]:
List(1, 2, 3) >| "x"

res9: List[String] = List("x", "x", "x")

In [11]:
List(1, 2, 3) as "x"

res10: List[String] = List("x", "x", "x")

In [12]:
List(1, 2, 3).fpair

res11: List[(Int, Int)] = List((1, 1), (2, 2), (3, 3))

In [13]:
List(1, 2, 3).strengthL("x")

res12: List[(String, Int)] = List(("x", 1), ("x", 2), ("x", 3))

In [14]:
List(1, 2, 3).strengthR("x")

res13: List[(Int, String)] = List((1, "x"), (2, "x"), (3, "x"))

In [15]:
List(1, 2, 3).void

res14: List[Unit] = List((), (), ())

## Applicative

In [15]:
List(1, 2, 3, 4) map {(_: Int) * (_:Int)}

cmd15.sc:1: type mismatch;
 found   : (Int, Int) => Int
 required: Int => ?
val res15 = List(1, 2, 3, 4) map {(_: Int) * (_:Int)}
                                           ^

: 

In [16]:
val res = List(1, 2, 3, 4) map {(_: Int) * (_:Int)}.curried

res: List[Int => Int] = List(
  scala.Function2$$Lambda$540/318755294@2d3e2ac9,
  scala.Function2$$Lambda$540/318755294@7c0afa29,
  scala.Function2$$Lambda$540/318755294@4d140ccc,
  scala.Function2$$Lambda$540/318755294@e5a7767
)

In [17]:
res map {_(9)}

res16: List[Int] = List(9, 18, 27, 36)

```scala
trait Applicative[F[_]] extends Apply[F] { self =>
  def point[A](a: => A): F[A]

  /** alias for `point` */
  def pure[A](a: => A): F[A] = point(a)

  ...
}
```

In [18]:
1.point[List]

res17: List[Int] = List(1)

In [19]:
1.point[Option]

res18: Option[Int] = Some(1)

In [20]:
1.point[Option] map {_ + 2}

res19: Option[Int] = Some(3)

In [21]:
1.point[List] map {_ + 2}

res20: List[Int] = List(3)

## Apply

```scala
trait Apply[F[_]] extends Functor[F] { self =>
  def ap[A,B](fa: => F[A])(f: => F[A => B]): F[B]
}
```

In [22]:
9.some <*> {(_: Int) + 3}.some

res21: Option[Int] = Some(12)

In [23]:
1.some <* 2.some

res22: Option[Int] = Some(1)

In [24]:
none <* 2.some

res23: Option[Nothing] = None

In [25]:
1.some *> 2.some

res24: Option[Int] = Some(2)

In [26]:
none *> 2.some

res25: Option[Int] = None

## Apply としての Option

In [27]:
9.some <*> {(_: Int) + 3}.some

res26: Option[Int] = Some(12)

In [28]:
3.some <*> { 9.some <*> {(_: Int) + (_: Int)}.curried.some }

res27: Option[Int] = Some(12)

In [29]:
^(3.some, 5.some) {_ + _}

res28: Option[Int] = Some(8)

In [30]:
^(3.some, none[Int]) {_ + _}

res29: Option[Int] = None

In [31]:
(3.some |@| 5.some) {_ + _}

res30: Option[Int] = Some(8)

## Apply としての List

In [32]:
List(1, 2, 3) <*> List((_: Int) * 0, (_: Int) + 100, (x: Int) => x * x)

res31: List[Int] = List(0, 0, 0, 101, 102, 103, 1, 4, 9)

In [33]:
List(3, 4) <*> { List(1, 2) <*> List({(_: Int) + (_: Int)}.curried, {(_: Int) * (_: Int)}.curried) }

res32: List[Int] = List(4, 5, 5, 6, 3, 4, 6, 8)

In [34]:
(List("ha", "heh", "hmm") |@| List("?", "!", ".")) {_ + _}

res33: List[String] = List("ha?", "ha!", "ha.", "heh?", "heh!", "heh.", "hmm?", "hmm!", "hmm.")

## Zip List

In [35]:
val res = streamZipApplicative.ap(Tags.Zip(Stream(1, 2))) (Tags.Zip(Stream({(_: Int) + 3}, {(_: Int) * 2})))

res: {type Tag = scalaz.Tags.Zip;type Self = scala.collection.immutable.Stream[Int]} = Stream(4, ?)

In [36]:
Tag.unwrap(res).toList

res35: List[Int] = List(4, 4)

## Applicative の便利な関数

In [37]:
val res = Apply[Option].lift2((_: Int) :: (_: List[Int]))

res: (Option[Int], Option[List[Int]]) => Option[List[Int]] = scalaz.Apply$$Lambda$733/1543923104@cacd4d5

In [38]:
res(3.some, List(4).some)

res37: Option[List[Int]] = Some(List(3, 4))

In [39]:
def sequenceA[F[_]: Applicative, A](list: List[F[A]]): F[List[A]] = list match {
    case Nil     => (Nil: List[A]).point[F]
    case x :: xs => (x |@| sequenceA(xs)) {_ :: _} 
}

defined function sequenceA

In [40]:
sequenceA(List(1.some, 2.some))

res39: Option[List[Int]] = Some(List(1, 2))

In [41]:
sequenceA(List(3.some, none, 1.some))

res40: Option[List[Int]] = None

In [42]:
sequenceA(List(List(1, 2, 3), List(4, 5, 6)))

res41: List[List[Int]] = List(
  List(1, 4),
  List(1, 5),
  List(1, 6),
  List(2, 4),
  List(2, 5),
  List(2, 6),
  List(3, 4),
  List(3, 5),
  List(3, 6)
)

In [43]:
type Function1Int[A] = ({type l[A]=Function1[Int, A]})#l[A]

defined type Function1Int

In [44]:
val res = sequenceA(List((_: Int) + 3, (_: Int) + 2, (_: Int) + 1): List[Function1Int[Int]])

res: Int => List[Int] = scalaz.std.FunctionInstances$$anon$3$$Lambda$3014/956417547@c00ed37

In [45]:
res(3)

res44: List[Int] = List(6, 5, 4)